# Nandu - "Echo" Process

In [1]:
import requests
import json
import time

In [2]:
BASE_URL = "http://localhost:5000"

## Show all processes

In [3]:
resp = requests.get(f"{BASE_URL}/processes")
resp.ok

True

In [4]:
resp.json()

{'processes': [{'version': '0.2.0',
   'id': 'hello-world',
   'title': 'Hello World',
   'description': 'An example process that takes a name as input, and echoes it back as output. Intended to demonstrate a simple process with a single literal input.',
   'jobControlOptions': ['sync-execute', 'async-execute'],
   'keywords': ['hello world', 'example', 'echo'],
   'links': [{'type': 'text/html',
     'rel': 'about',
     'title': 'information',
     'href': 'https://example.org/process',
     'hreflang': 'en-US'},
    {'type': 'application/json',
     'rel': 'self',
     'href': 'http://localhost:5000/processes/hello-world?f=json',
     'title': 'Process description as JSON',
     'hreflang': 'en-US'},
    {'type': 'text/html',
     'rel': 'alternate',
     'href': 'http://localhost:5000/processes/hello-world?f=html',
     'title': 'Process description as HTML',
     'hreflang': 'en-US'},
    {'type': 'text/html',
     'rel': 'http://www.opengis.net/def/rel/ogc/1.0/job-list',
     'hr

## Show description of "Echo" process

In [5]:
resp = requests.get(f"{BASE_URL}/processes/echo")
resp.ok

True

In [6]:
resp.json()

{'id': 'echo',
 'title': 'Echo Process',
 'description': 'Testable Echo process.',
 'version': '1.0.0',
 'jobControlOptions': ['sync-execute', 'async-execute'],
 'outputTransmission': ['value'],
 'inputs': {'echoInput': {'title': 'Echo value',
   'description': 'Value to be echoed back.',
   'minOccurs': 1,
   'maxOccurs': 1,
   'schema': {'type': 'string', 'enum': ['Echo', 'Test', '42']}},
  'pause': {'title': 'Pause value',
   'description': 'Value to control the processing time.',
   'minOccurs': 1,
   'maxOccurs': 1,
   'schema': {'type': 'number', 'enum': [5.5, 10.25, 42.0]}}},
 'outputs': {'echoOutput': {'schema': {'type': 'string'}}},
 'links': [{'type': 'text/html',
   'rel': 'about',
   'title': 'information',
   'href': 'https://example.org/process',
   'hreflang': 'en-US'},
  {'type': 'application/json',
   'rel': 'self',
   'href': 'http://localhost:5000/processes/echo?f=json',
   'title': 'Process description as JSON',
   'hreflang': 'en-US'},
  {'type': 'text/html',
   'r

## Execute "Echo" process - sync mode

In [7]:
data = {
    "inputs": {
        "echoInput": "Sun rise",
        "pause": 5.5
    }
}

headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

In [8]:
resp = requests.post(f"{BASE_URL}/processes/echo/execution", data=json.dumps(data), headers=headers)
resp.ok

True

In [9]:
resp.json()

{'id': 'echoOutput', 'value': 'Sun rise'}

## Execute "Echo" process - async mode

In [10]:
data = {
    "inputs": {
        "echoInput": "Sun rise",
        "pause": 5.5
    } 
}

headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Prefer": "respond-async"
}

In [11]:
resp = requests.post(f"{BASE_URL}/processes/echo/execution", data=json.dumps(data), headers=headers)
resp.ok

True

In [12]:
resp.json()

In [13]:
job_url = resp.headers["Location"]
job_url

'http://localhost:5000/jobs/04ff4fd4-7422-11ef-a674-acde48001122'

In [14]:
# poll job status until finished

while True:
    resp = requests.get(job_url)
    status = resp.json()['status']
    progress = resp.json()['progress']
    message = resp.json()['message']
    start = resp.json()['job_start_datetime']
    end = resp.json()['job_end_datetime']

    print(f"status={status}, progress={progress}, message={message}, start={start}, end={end}")

    if status in ["successful"]:
        break

    time.sleep(2)

status=accepted, progress=5, message=Job accepted and ready for execution, start=2024-09-16T11:51:33.332788Z, end=None
status=accepted, progress=5, message=Job accepted and ready for execution, start=2024-09-16T11:51:33.332788Z, end=None
status=accepted, progress=5, message=Job accepted and ready for execution, start=2024-09-16T11:51:33.332788Z, end=None
status=successful, progress=100, message=Job complete, start=2024-09-16T11:51:33.332788Z, end=2024-09-16T11:51:38.846382Z


In [15]:
# show detailed job status
resp.json()

{'type': 'process',
 'processID': 'echo',
 'jobID': '04ff4fd4-7422-11ef-a674-acde48001122',
 'status': 'successful',
 'message': 'Job complete',
 'progress': 100,
 'parameters': None,
 'job_start_datetime': '2024-09-16T11:51:33.332788Z',
 'job_end_datetime': '2024-09-16T11:51:38.846382Z',
 'links': [{'href': 'http://localhost:5000/jobs/04ff4fd4-7422-11ef-a674-acde48001122/results?f=html',
   'rel': 'http://www.opengis.net/def/rel/ogc/1.0/results',
   'type': 'text/html',
   'title': 'Results of job as HTML'},
  {'href': 'http://localhost:5000/jobs/04ff4fd4-7422-11ef-a674-acde48001122/results?f=json',
   'rel': 'http://www.opengis.net/def/rel/ogc/1.0/results',
   'type': 'application/json',
   'title': 'Results of job as JSON'}]}

In [16]:
# show job result

resp = requests.get(f"{job_url}/results?f=json")
resp.json()

{'id': 'echoOutput', 'value': 'Sun rise'}